# Discrete Polar Lattice Validation (ℓ_max = 2)

This notebook demonstrates the core numerical claims:

1. **Exact SU(2) commutators**: $[L_x, L_y] = i L_z$ (and cyclic) to machine precision
2. **Correct L² spectrum**: Eigenvalues $\ell(\ell+1)$ with degeneracies $2(2\ell+1)$
3. **Geometric construction**: Simple rules for lattice points and quantum numbers

**Reproducibility**: All results can be verified by running this notebook.

In [ ]:
import sys
import os

# Add src to path
sys.path.insert(0, os.path.join(os.path.dirname(os.path.abspath('__file__')), '..', 'src'))

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle

from lattice import build_lattice, count_degeneracies
from operators import build_angular_momentum_operators
from validation import (
    validate_commutators, 
    validate_spectrum,
    print_commutator_results,
    print_spectrum_results
)

# Plotting settings
plt.rcParams['figure.figsize'] = (10, 8)
plt.rcParams['font.size'] = 11

## 1. Build the Lattice

The lattice is constructed using geometric rules:
- Ring radius: $r_\ell = 1 + 2\ell$
- Points per ring: $N_\ell = 2(2\ell + 1)$
- Quantum number assignment: $(\ell, m_\ell, m_s)$ with spin interleaving

In [ ]:
# Build lattice for ℓ_max = 2
ℓ_max = 2
lattice = build_lattice(ℓ_max=ℓ_max)

print(f"Lattice built: ℓ_max = {ℓ_max}")
print(f"Total points: {lattice['n_points']}")
print()

# Show degeneracies
degeneracies = count_degeneracies(lattice)
print("Degeneracies by ℓ:")
for ℓ, deg in degeneracies.items():
    expected = 2 * (2 * ℓ + 1)
    print(f"  ℓ = {ℓ}: {deg} states (expected {expected})")

## 2. Visualize the Lattice

Plot the 2D lattice structure, colored by ℓ and spin.

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 7))

# Plot 1: Color by ℓ
points = lattice['points']
ell = lattice['ell']

scatter1 = ax1.scatter(points[:, 0], points[:, 1], c=ell, cmap='viridis', 
                      s=150, alpha=0.7, edgecolors='black', linewidth=0.5)
ax1.set_aspect('equal')
ax1.set_xlabel('x', fontsize=13)
ax1.set_ylabel('y', fontsize=13)
ax1.set_title('Lattice colored by ℓ', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
plt.colorbar(scatter1, ax=ax1, label='ℓ')

# Draw rings
for ℓ in range(ℓ_max + 1):
    r_ℓ = 1 + 2 * ℓ
    circle = Circle((0, 0), r_ℓ, fill=False, edgecolor='gray', 
                   linestyle='--', linewidth=1, alpha=0.5)
    ax1.add_patch(circle)

# Plot 2: Color by spin
m_s = lattice['m_s']
colors = ['red' if s > 0 else 'blue' for s in m_s]

ax2.scatter(points[:, 0], points[:, 1], c=colors, 
           s=150, alpha=0.7, edgecolors='black', linewidth=0.5)
ax2.set_aspect('equal')
ax2.set_xlabel('x', fontsize=13)
ax2.set_ylabel('y', fontsize=13)
ax2.set_title('Lattice colored by spin', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Legend for spin
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='red', label='$m_s = +1/2$'),
    Patch(facecolor='blue', label='$m_s = -1/2$')
]
ax2.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.savefig('figures/lattice_structure.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved: figures/lattice_structure.png")

## 3. Build Angular Momentum Operators

Construct $L_z$, $L_\pm$, $L_x$, $L_y$, and $L^2$ using standard angular momentum algebra.

In [ ]:
print("Building operators...")
ops = build_angular_momentum_operators(lattice)

print("Operators built:")
for name, op in ops.items():
    print(f"  {name}: {op.shape} sparse matrix, {op.nnz} non-zero elements")

## 4. Validate SU(2) Commutation Relations

Test: $[L_x, L_y] = i L_z$, $[L_y, L_z] = i L_x$, $[L_z, L_x] = i L_y$

In [ ]:
# Validate commutators
commutator_results = validate_commutators(ops, tolerance=1e-12)

# Print formatted results
print_commutator_results(commutator_results)

## 5. Validate L² Eigenvalue Spectrum

Check that $L^2$ has eigenvalues $\ell(\ell+1)$ with degeneracies $2(2\ell+1)$.

In [ ]:
# Validate spectrum
spectrum_results = validate_spectrum(lattice, ops, tolerance=1e-10)

# Print formatted results
print_spectrum_results(spectrum_results)

## 6. Visualize Operator Matrices

Show the structure of $L_z$, $L_+$, and $L^2$ as matrices.

In [ ]:
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

operators_to_plot = [
    ('Lz', 'L_z'),
    ('Lplus', 'L_+'),
    ('Lx', 'L_x'),
]

for idx, (key, label) in enumerate(operators_to_plot):
    op = ops[key]
    op_dense = op.toarray()
    
    # Real part
    ax = axes[0, idx]
    im = ax.imshow(np.real(op_dense), cmap='RdBu', aspect='auto',
                  vmin=-np.abs(op_dense).max(), vmax=np.abs(op_dense).max())
    ax.set_title(f'{label} (Real)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    plt.colorbar(im, ax=ax, fraction=0.046)
    
    # Imaginary part
    ax = axes[1, idx]
    im = ax.imshow(np.imag(op_dense), cmap='RdBu', aspect='auto',
                  vmin=-np.abs(op_dense).max(), vmax=np.abs(op_dense).max())
    ax.set_title(f'{label} (Imaginary)', fontsize=12, fontweight='bold')
    ax.set_xlabel('Column')
    ax.set_ylabel('Row')
    plt.colorbar(im, ax=ax, fraction=0.046)

plt.tight_layout()
plt.savefig('figures/operator_matrices.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved: figures/operator_matrices.png")

## 7. Eigenvalue Distribution

Plot the distribution of $L^2$ eigenvalues.

In [ ]:
L2 = ops['L2']
L2_diag = np.real(L2.diagonal())

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram of eigenvalues
ax1.hist(L2_diag, bins=20, edgecolor='black', alpha=0.7)
ax1.set_xlabel('$L^2$ eigenvalue', fontsize=13)
ax1.set_ylabel('Count', fontsize=13)
ax1.set_title('Distribution of $L^2$ eigenvalues', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Expected values
expected_vals = [0, 2, 6]
for val in expected_vals:
    ax1.axvline(val, color='red', linestyle='--', linewidth=2, alpha=0.7)

# Scatter plot: eigenvalue vs state index
ax2.scatter(range(len(L2_diag)), L2_diag, c=lattice['ell'], 
           cmap='viridis', s=100, alpha=0.7, edgecolors='black', linewidth=0.5)
ax2.set_xlabel('State index', fontsize=13)
ax2.set_ylabel('$L^2$ eigenvalue', fontsize=13)
ax2.set_title('$L^2$ eigenvalues by state', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

# Expected horizontal lines
for ℓ in range(ℓ_max + 1):
    expected = ℓ * (ℓ + 1)
    ax2.axhline(expected, color='red', linestyle='--', linewidth=1, alpha=0.5)
    ax2.text(lattice['n_points'] + 0.5, expected, f"ℓ={ℓ}", 
            fontsize=10, va='center', color='red')

plt.tight_layout()
plt.savefig('figures/eigenvalue_spectrum.png', dpi=150, bbox_inches='tight')
plt.show()

print("Figure saved: figures/eigenvalue_spectrum.png")

## Summary

This notebook demonstrates that the discrete polar lattice:

✅ **Exactly reproduces SU(2) commutation relations** to machine precision (< 10⁻¹²)  
✅ **Has correct L² spectrum** with eigenvalues ℓ(ℓ+1) and degeneracies 2(2ℓ+1)  
✅ **Follows simple geometric rules** for lattice construction  

**Reproducibility**: All numerical claims can be verified by running this notebook or the test suite (`pytest tests/`).